In [ ]:
from git_request import search_top_starred_repositories, search_top_related_local_repositories_with_cs, search_top_related_local_repositories_with_chroma
from gpt_request import get_response_from_chatgpt_with_context
from generate_local_github_database import download_and_save_git_stared_reposiories_according_to_user, download_and_save_git_reposiories_according_to_keyword
from generate_local_openml_database import download_and_save_openml_dataset_infomration_with_openml, download_and_save_openml_dataset_infomration_with_request
from bs4 import BeautifulSoup

import os
import pandas as pd
import requests

In [ ]:
import sdsc_cataglog

In [ ]:
prompt = f"What is the theme studied in the following use case, please answer only with a keyword less then 20 letter: {use_case}"
context = []
response, context = get_response_from_chatgpt_with_context(prompt, context)
response

# 示例
场景与动机

功能：客户通过文本对面临的挑战进行描述。我们将对这些文本描述进行总结，并基于此，推荐合适的工具和相关的数据集。


In [3]:
from constants import CHROMA_SETTINGS, SOURCE_DIRECTORY, PERSIST_DIRECTORY
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
import torch

from localgpt_request import load_model_for_text_generation,load_model_for_QA

/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load model
# local_llm = load_model_for_QA()
local_llm = load_model_for_text_generation()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.94s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
# wrap model
template = """Question: {question}
    Answer: Let's think step by step."""
promptT = PromptTemplate(template = template, input_variables=["question"])
llm_chain = LLMChain(prompt=promptT, llm=local_llm)

In [6]:
# use case
use_case = "Diese Aufgabe war mit zwei Schwierigkeiten verbunden: 1) die Anforderung der Interpretierbarkeit schränkt die Auswahl der Modelle ein und 2) die große Anzahl als Text gespeicherter Merkmale erschwert die Analyse und beeinträchtigt gleichzeitig die Leistung einiger Modelle.Es gibt viele komplexe Modelle, deren Entscheidungen schwer zu erklären sind, tiefe neuronale Netze sind eines der typischen Beispiele dafür. Attribute, die mit Hilfe von Text gespeichert werden, sind meist kontextabhängig und können durch semantische Analyse gut behandelt werden, aber sie erfordert eine große Menge Daten und Zeit und ist nicht interpretierbar. Stattdessen können bei einigen einfachen Methoden Informationen hinzugefügt oder entfernt werden."

In [7]:
#wrap promt, replace with function later
promt = "What is the theme studied in the following use case, please answer only with a keyword less then 20 letter:" + use_case
o = llm_chain.run(promt).strip()
o


/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


'The first difficulty mentioned is that the requirement of interpretable models limits the selection of models. This means we need to find a model that can explain its decision-making process clearly and easily. The second difficulty is that there are many complex models out there, which makes it difficult to understand their decisions. Deep neural networks are one example of such models. They have many layers and are hard to interpret. In this context, "interpreting" means understanding how the model arrived at its final output or prediction. To solve these difficulties, we might consider using simpler models that are easier to interpret, like logistic regression or linear regression. We could also try feature engineering techniques to extract more meaningful features from our data instead of relying on text-based attributes.'

In [8]:
# analyse problem and get three problems
promt = f"According to the given description, what is the main problems faced by this study, please answer with 3 lines, each line represents a problem." + use_case
o = llm_chain.run(promt).strip()
o

'The first difficulty is that some models are not interpretable because of the requirement for interpretability. This means that we cannot easily understand how these models make their predictions and decisions. For example, deep neural networks can be very complex and difficult to explain. The second difficulty is that there are many text-based features stored as text, which makes it harder to analyze them and affects performance at the same time. These attributes may depend on context and can be analyzed using semantic analysis, but they require a large amount of data and time and are not directly interpretable. Instead, simpler methods may be used to add or remove information from the data.'

In [ ]:
# summary the problems 
local_llm = load_model_for_text_generation
template = """Please summarize the following text"""
promptT = PromptTemplate(template = template, input_variables=["question"])
llm_sum = LLMChain(prompt=promptT, llm=local_llm)

o = llm_chain.run(o).strip()
o

In [ ]:
# generate embedding
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                           model_kwargs={"device": device})

embd_o = embeddings.embed_query(o)


In [8]:
# looking for tool that can deal with the problem
o = search_top_related_local_repositories_with_cs(embd_o)
o

In [ ]:
# looking for dataset that may be helpful to deal with the problem
o = search_top_related_local_datasets_with_chatgpt(embd_o)
o

为了在本地部署llm，我们需要从huggingface选择一个预训练好的模型。从零开始训练一个语言模型需要大量的文本和训练时间。

### pipeline of the app
- summarize the given challenge 
- conclude three topics discuss

### what is NLP and what is NLP for
NLP常见的任务包括
- 特征提取: feasture-extraction
- 情感分析: sentiment-analysis
- QA: question-answering
- fill-mask
- summarization
- translation_xx_to_yy
- text-generation

### why do we need NLP
...


针对我们的问题，我们需要用到：
- QA：llm通常在处理大量的信息的时候会出现局部偏好。这意味这它将提取局部的信息，并根据这些信息进行回复，从而导致信息的丢失。为此，我们需要根据需求，对问题进行分解，然后以渐进的方法和llm进行交互并获取我们需要的信息。为此，我们需要QA功能。
- summarization: 根据我们的任务，我们需要对输入的文本进行总结，除此之外，在提问的过程中，我们也需要通过总结来精简我们的文本内容，从而在降低QA压力的同时，避开QA的字数限制。另外，虽然在一些情况下QA可以给出对特定任务的简介答案，从而看起来似乎实现了‘总结’的功能。但总结和QA其实是两种截然不同的自然语言处理任务，他们的目标和处理方式有本质的区别。和QA相比，总结需要理解整个文本 的主要内容和结构，并在输出摘要中保留这些信息。QA更多是一个相似度匹配的过程。这种区别也体现在，QA处理大量信息时容易出现偏好的情况。
- feature extraction: to generate word embedding


- QA: llm usually has a local preference when processing large amounts of information. This means that it will extract localised information and reply based on that information, leading to loss of information. For this reason, we need to decompose the problem based on the requirements and then interact with llm in an incremental way and get the information we need. For this, we need QA functionality.
- summarization: Depending on our task, we need to summarise the input text, and in addition to that, during the questioning process, we also need to streamline our text by summarising it, so as to avoid the QA word limit while reducing the QA pressure. Also, although in some cases QA can give a brief answer to a specific task, thus seemingly fulfilling the function of 'summarising'. However, summarisation and QA are in fact two very different natural language processing tasks, and their goals and processing styles are fundamentally different. In contrast to QA, which is more of a similarity matching process, summarisation involves understanding the main content and structure of the text and preserving this information in the output summary. QA is more of a similarity matching process. This difference is also reflected in the fact that QA is prone to preferences when dealing with large amounts of information.
- feature extraction: to generate word embedding


### consistent return
另外，我们可以观察到，当我们使用chatgpt的时候，尽管输入是相同的，但是得到的反馈常常是有区别的。这是由于llm模型在选择回复使用的词的时候，通过会通过参数调大低频单词被使用的可能性，从而提高了文本的多样性。然而这种情况我们是不需要的，或者更进一步，我们应该抑制这种情况的发生。在llm这个控制多样性的参数被命名为热度。热度越大越支持多样性。

In addition, we can observe that when we use chatgpt, the feedback we get is often different even though the inputs are the same. This is due to the fact that the llm model improves the diversity of the text when selecting the words to be used in the replies, by will parameterise the likelihood of low-frequency words being used. However this is a situation we do not need, or furthermore, we should suppress it. The parameter that controls diversity in llm is named heat. The higher the heat the more diversity is supported.


### what is token
在使用chatgpt的时候，我们还经常会听到一个词，token。往往我们能看到这样的限制，chatgpt 3.5支持的最大token数是4096. 这里的token并不是特指单词或字母。事实上，在我们把文本放入到llm之前，我们通常会进行一个额外的操作，即tokenizer。它负责将输入的文本拆分成更小的单元，通常是单词、子词或符号，以便模型可以理解和处理这些文本。不同的tokenizer的分词方法是存在区别。"Hello, world!"一般被分割为["Hello", ",", "world", "!"]。当然，tokenzier的功能远不止于分词，它同时还实现了很多其他的功能，比如，添加特殊符号，处理空格标点等。


When using chatgpt, we also often hear the word, token. often we can see the limitation that the maximum number of tokens supported by chatgpt 3.5 is 4096. token here doesn't mean words or letters in particular. In fact, before we put the text into llm, we usually perform an additional operation called tokenizer. it is responsible for splitting the input text into smaller units, usually words, sub-words or symbols, so that the model can understand and process them. There is a difference in the way different tokenizers split words. Hello, world!" is typically split into ["Hello", ",", "world", "!"] . Of course, tokenzier does much more than just split words, it also implements many other features, such as adding special symbols, handling spaces and punctuation, and so on.

### setup local llm
我们通过加载预训练的模型来搭建本地的llm。一般的llm模型都很大需要大量的文本进行训练。。。

We build a local llm by loading a pre-trained model. general llm models are large require a large amount of text for training.

### tool
github是一个。。。，特别是现在大部分论文都和github绑定。通过github我们能够获得最新的工具。基于此，我们使用github作为我们的工具库。然而，虽然github提供了api，他们提供的关键字机制，可以从标题，github关键字和readme中进行搜索。然而，这些内容都是主观编辑的，而且相同的对象存在多种不同的描述方式。使用关键字搜索，可能出现虽然文本高度相关，但是关键字没有出现的情况。特别是在出现专业用语和一般用于偏差的时候，这种情况更加普遍。比如，Unsupervised domain adajustation是跨领域技术的一个主要研究领域。但是对于新手来说一般会使用transfer learning对这个任务进行描述。他们的描述是正确的，不过范围太广。基于此，我们使用word embedding来搜索工具。因为github的库都有read么文件。用来说明这个github的主要用途。类似于我们上面的操作，我们对这些文本进行总结，并通过embedding生成他们的描述。并在local进行存储。

当新的访问到达时，我们适用访问的embedding和我们本地保存的embedding进行对比，找到最接近的对应工具。

Github is a.... , especially now that most papers are tied to github. Through github we are able to get the latest tools. For this reason, we use github as our tool repository. However, although github provides api, they provide keyword mechanism to search from title, github keywords and readme. However, these are subjectively edited and there are many different ways of describing the same object. Using keyword searches, there may be cases where keywords do not appear even though the text is highly relevant. This is especially common when there are specialised terms and general use for deviations. For example, Unsupervised domain adajustation is a major research area in cross-domain technologies. However, novices generally use transfer learning to describe this task. Their description is correct, however the scope is too broad. Based on this, we use word embedding to search for tools. Because github libraries have read files. It is used to describe the main purpose of this github. Similar to what we did above, we summarise these texts and generate their descriptions through embedding. And store them locally.

When a new access arrives, we compare the embedding of the access with our locally stored embedding to find the closest corresponding tool.

### openml
和tool相似，我们也为openml建立本地数据库，然而使用embedding进行搜索。

Similarly to tool, we build a local database for openml, however use embedding for searching.

When a new access arrives, we compare the embedding of the access with our locally stored embedding to find the closest corresponding tool.